In [1]:
import pickle
import numpy as np
from sklearn.manifold import TSNE

from plotly import tools
from matplotlib import cm
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import os
import glob
from os import listdir
from os.path import isfile, join

In [2]:
def files_list(path):
    return [f for f in glob.glob(path + '*.csv')]


def build_bank(path, f_names = []):
    files = files_list(path)

    features_bank = []

    for feature_file in files:
        
        if f_names != [] and \
            os.path.basename(feature_file)[:-4] not in f_names:
                continue
        
        feature_table = open(feature_file, "r").readlines()[1:]
        feature_values = []

        for line in feature_table:
            feature_values.append(line.split(","))

        features_bank.append(feature_values)
        n_songs = len(feature_values)
        
    f_normal = []    
    for song_idx in range(n_songs):
        fs = features_bank[0][song_idx][0:2]
        for feature in features_bank:
            fs += feature[song_idx][2:]

        f_normal.append(list(map(lambda x: x.strip(), fs)))

    return f_normal

In [3]:
f_names = ["liveness","instrumentalness", "danceability","energy", "speechiness", "popularity","tempo","valence"]

data = build_bank("../data/features/favourite/")

f_art = [x[0] for x in data]
f_vals = [x[2:] for x in data]
f_labels = [x[0] + " " + x[1] for x in data]

data1 = build_bank("../data/features/tags/")

f_art1 = [x[0] for x in data1]
f_vals1 = [x[2:] for x in data1]
f_labels1 = [x[0] + " " + x[1] for x in data1]

In [ ]:
max_vals = []
def add_bar(fig, inp, n_row, n_col):
    global chart_count

    x, y = np.histogram(inp, bins=20)
    max_vals.append(max(x))
    data = go.Bar(
            x=y,
            y=x,
            name="",
            marker = dict(color='green'),
            opacity=0.7
    )
    fig.append_trace(data, (chart_count // n_col) + 1, (chart_count % n_col) + 1)
    fig['layout']['yaxis{0}'.format(chart_count+1)].update(ticklen=3)
    fig['layout']['yaxis{0}'.format(chart_count+1)]['tickfont'].update(size=11,
                                                                       color="lightgrey")
    fig['layout']['xaxis{0}'.format(chart_count+1)].update(range=[min(y),max(y)])
    chart_count += 1

def add_line(fig, inp, n_row, n_col):
    global chart_count

    xh, yh = np.histogram(list(map(float, inp)), bins=20)
    
    data = go.Scatter(
            x=yh,
            y=max_vals[chart_count] * np.array(xh) / max(xh),#max_vals[chart_count],
            name="",
            marker = dict(color='blue'),
            opacity=0.7,
            mode='lines',
            hoverinfo='none',
            line=dict(shape='spline')
            )
    fig.append_trace(data, (chart_count // n_col) + 1, (chart_count % n_col) + 1)
    fig['layout']['yaxis{0}'.format(chart_count+1)].update(ticklen=3)
    fig['layout']['yaxis{0}'.format(chart_count+1)]['tickfont'].update(size=11,
                                                                       color="lightgrey")
    chart_count += 1
    
def draw_bars():
    global chart_count
    n_row = 4
    n_col = 2
    n_charts = n_row * n_col

    fig = tools.make_subplots(rows=n_row, cols=n_col, print_grid=False,
                              subplot_titles=fl)
    cl = []
    for feature in fl:
        cl.append(pickle.load(open("../data/regressors/{}".format(feature), 'rb')))
    
    xl = []
    for clf in cl:
        xl.append(clf.predict(f_vals))
    
    for x in xl:
        add_bar(fig, x, n_row, n_col)

    chart_count = 0
        
    xl = []
    for clf in cl:
        xl.append(clf.predict(f_vals1))
        
    for x in xl:
        add_line(fig, x, n_row, n_col)
        
    fig['layout'].update(height=750, width=750, showlegend=False)
    py.iplot(fig, show_link=False)
  
fl = ["liveness","instrumentalness", "danceability","energy", "speechiness", "popularity","tempo","valence"]
chart_count = 0
draw_bars()

In [ ]:
fl=["liveness","instrumentalness", "danceability","energy", "speechiness", "popularity","tempo","valence"]
cl = []

for feature in fl:
    cl.append(pickle.load(open("../data/regressors/{}".format(feature), 'rb')))

nf = len(fl)
data = build_bank("../data/features/favourite/")

f_vals = [x[2:] for x in data]

xl = []
for clf in cl:
    xl.append(clf.predict(f_vals))

m = np.zeros((nf,nf))

for x in range(nf):
    for y in range(nf):
        m[x, y] = np.corrcoef(xl[x], xl[y])[0][1]
m = np.round(m,3)
        
from matplotlib import cm
def color_convert(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []
    
    for k in range(pl_entries):
        C = np.array(cmap(k * h)[:3]) * 255
        pl_colorscale.append([k * h, 'rgb' + str((C[0], C[1], C[2]))])
        
    return pl_colorscale

blues_cmap = cm.get_cmap('Blues')
blues = color_convert(blues_cmap, 255)        

trace = go.Heatmap(x=fl,
                   y = fl,
                   z = m,
                   colorscale = blues,
                   showscale = False)

layout = go.Layout(
    width = 500,
    height = 500)

data=[trace]
py.iplot({'data': data, 'layout': layout}, show_link=False)

In [ ]:
fl=["liveness","instrumentalness", "danceability","energy", "speechiness", "popularity","tempo","valence"]

def clf_bank():
    cb = []
    for feature in fl:
        cb.append(pickle.load(open("../data/regressors/{}".format(feature), 'rb')))
    return cb

def pred_values(cb, features_vals):
    pv = []
    for clf in cb:
        pv.append(clf.predict(features_vals))
    return pv
        
def color_convert(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []
    
    for k in range(pl_entries):
        C = np.array(cmap(k * h)[:3]) * 255
        pl_colorscale.append([k * h, 'rgb' + str((C[0], C[1], C[2]))])
        
    return pl_colorscale

def scatter1(fig, x, y, title1, title2):
    global chart_count
    blues_cmap = cm.get_cmap('Blues')
    blues = color_convert(blues_cmap, 255)

    H, xedges, yedges = np.histogram2d(x, y)

    data = go.Heatmap(x = xedges,
                       y = xedges,
                       z = H,
                       colorscale = blues,
                       showscale = False)

    layout = go.Layout(
        width = 100,
        height = 100,
        xaxis=dict(title = f1),
        yaxis=dict(title = f2))

    fig.append_trace(data, (chart_count // n_col) + 1, (chart_count % n_col) + 1)
    fig['layout'].update(width = 600, height = 1200)
    fig['layout']['xaxis{0}'.format(chart_count+1)].update(title = title2) 
    fig['layout']['yaxis{0}'.format(chart_count+1)].update(title = title1) 
    
    chart_count += 1
    
    return fig

pairs = [["danceability", "valence"],
         ["danceability", "popularity"],
         ["danceability", "tempo"],
         ["danceability", "energy"],
         ["popularity", "valence"],
         ["popularity", "instrumentalness"],
         ["popularity", "energy"],
         ["energy", "speechiness"]]

n_row = len(pairs) // 2
n_col = 2
n_charts = n_row * n_col
chart_count = 0
    
fig = tools.make_subplots(rows = n_row, 
                          cols = n_col, 
                          print_grid = False, 
                          vertical_spacing = 0.1,
                          horizontal_spacing = 0.2,
                          subplot_titles = [])
cb = clf_bank()
pv = pred_values(cb, f_vals)

for pair in pairs:
    idx1 = fl.index(pair[0])
    idx2 = fl.index(pair[1])

    x = pv[idx1]
    y = pv[idx2]

    fig = scatter1(fig, x, y, pair[0], pair[1])
        
py.iplot(fig, show_link=False)

In [ ]:
def features_scatter(f1, f2, artist = "", color="red"):
    clf1 = pickle.load(open("../data/regressors/{}".format(f1), 'rb'))
    clf2 = pickle.load(open("../data/regressors/{}".format(f2), 'rb'))

    x1 = clf1.predict(f_vals)
    x2 = clf2.predict(f_vals)

    data = [dict(
            type = 'scatter',
            x = [x for k, x in enumerate(x1) if f_art[k] == artist],
            y = [x for k, x in enumerate(x2) if f_art[k] == artist],
            hoverinfo = "text",
            hoveron = "points",
            mode = 'markers',
            text = [x for k, x in enumerate(f_labels) if f_art[k] == artist],
            marker = dict(color=color, symbol="circle-open"),
            name = artist,
            opacity=0.75
            ),
           dict(
            type = 'scatter',
            x = [x for k, x in enumerate(x1) if f_art[k] != artist],
            y = [x for k, x in enumerate(x2) if f_art[k] != artist],
            hoverinfo = "text",
            hoveron = "points",
            mode = 'markers',
            text = [x for k, x in enumerate(f_labels) if f_art[k] != artist],
            marker = dict(color=color),
            name = "Other",
            opacity=0.75
            )]

    layout = dict(
            hovermode = 'closest',
            xaxis=dict(title = f1),
            yaxis=dict(title = f2),
            )

    py.iplot({'data': data, 'layout': layout}, validate=False, show_link=False)

In [ ]:
features_scatter("danceability", "valence", artist = "The_xx", color = "red")

In [ ]:
features_scatter("danceability", "energy", artist = "The_xx", color = "green")

In [ ]:
features_scatter("valence", "energy", artist = "The_xx", color = "blue")